In [6]:
import datetime
import logging
import os
import glob 
from datetime import datetime, timedelta

from pathlib import Path
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from pyproj import Transformer
from sklearn.metrics import cohen_kappa_score
from shapely.geometry import LinearRing, Point, Polygon, box
import rasterio
import requests
from pprint import pprint


In [8]:
def download_dswx_hls(polys, 
                      datetime_start_str, 
                      datetime_end_str, 
                      MGRS_tile,
                      download_folder,
                      download_flag=True):
    CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
    url = f'{CMR_OPS}/{"granules"}'
    boundind_box = polys.bounds
    provider = 'POCLOUD'
    parameters = {'temporal': f'{datetime_start_str},{datetime_end_str}',
                    'concept_id': 'C2617126679-POCLOUD',
                    'provider': provider,
                    'bounding_box': f'{boundind_box[1]+0.2},{boundind_box[0]+0.2},{boundind_box[3]-0.2},{boundind_box[2]-0.2}',
                    'page_size': 200,}
    print(parameters)
    response = requests.get(url,
                            params=parameters,
                            headers={
                                'Accept': 'application/json'
                            }
                        )
    print(response.status_code)
    print(response.headers['CMR-Hits'])
    downloaded_list = []
    num_search_data = response.headers['CMR-Hits']
    number_hls_data = 0
    if num_search_data:
        collections = response.json()['feed']['entry']
        # print('collection', collections)
        for collection in collections:
            dswx_hls_file_id = collection['producer_granule_id']
            if MGRS_tile in dswx_hls_file_id:
                print('url', f'{collection["links"][4]["href"]}')
                print('s3', f'{collection["links"][3]["href"]}')
                if download_flag:
                    dswx_hls_url = collection["links"][4]["href"]
                    dswx_hls_filename = os.path.basename(dswx_hls_url)
                    download_file = f'{download_folder}/{dswx_hls_filename}'

                    if not os.path.isfile(download_file):
                        response = requests.get(dswx_hls_url, stream=True)

                    # Check if the request was successful
                        if response.status_code == 200:
                            # Open a local file with wb (write binary) permission.
                            with open(f'{download_file}', 'wb') as file:
                                print('downloading')
                                for chunk in response.iter_content(chunk_size=128):
                                    file.write(chunk)
                else:
                    print('under dev.')
                downloaded_list.append(download_file)
                number_hls_data += 1

            else:
                print('MGRS tile id does not match')

    return number_hls_data, downloaded_list


In [3]:
def download_dswx_s1(polys, datetime_start_str, datetime_end_str):
    CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
    url = f'{CMR_OPS}/{"granules"}'
    boundind_box = polys.bounds
    provider = 'POCLOUD'
    parameters = {'temporal': f'{datetime_start_str},{datetime_end_str}',
                    'concept_id': 'C2949811996-POCLOUD',
                    'provider': provider,
                    'bounding_box': f'{boundind_box[1]},{boundind_box[0]},{boundind_box[3]},{boundind_box[2]}',
                    'page_size': 200,}
    print(parameters)
    response = requests.get(url,
                            params=parameters,
                            headers={
                                'Accept': 'application/json'
                            }
                        )
    print(response.status_code)
    print(response.headers['CMR-Hits'])
    number_hls_data = response.headers['CMR-Hits']
    if number_hls_data:
        collections = response.json()['feed']['entry']

        for collection in collections:
            print(f'{collection["links"][4]["href"]}')
    return number_hls_data

In [9]:
poly_cand = box(-148, 64.90,  -147,65)
datetime_start_str = '2024-10-01T00:00:00Z'
datetime_end_str = '2024-10-13T00:00:00Z'
# formats_output = '%Y-%m-%dT%H:%M:%SZ'

download_dswx_hls(poly_cand, datetime_start_str, datetime_end_str)  

TypeError: download_dswx_hls() missing 2 required positional arguments: 'MGRS_tile' and 'download_folder'